In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [1]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# changing matplotlib the default style
matplotlib.style.use('ggplot')
#matplotlib.rcParams["figure.facecolor"] = "white"
#matplotlib.rcPar|ams["savefig.transparent"] = "True"


In [2]:
import pandas as pd


from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
from grids import *
from docking_analysis import *

from scipy import stats
import os
from efficacy_scripts import *




/home/enf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
from sklearn.preprocessing import scale

In [5]:
from detect_intermediates import *
from interpret_tICs import *

In [6]:
from msmbuilder.utils import verbosedump, verboseload


In [7]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import scale
from random import shuffle
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

In [8]:
def normalize_per_ligand(df):
    new_df = copy.deepcopy(df)
    n_df = new_df.ix[:, [i for i in range(0, new_df.shape[1]) if "difference" not in new_df.columns.values[i].lower()]]
    n_df = pd.DataFrame(preprocessing.scale(n_df.values, axis=1), index=n_df.index, columns=n_df.columns.values.tolist())
    new_df = pd.concat([new_df, n_df], axis=1)
    return(new_df)

In [9]:
def create_named_df(df, worker_pool):
    smiles_list, compound_names, pc_smiles, pc_names, pc_ids = convert_sdfs_to_compounds(["/home/enf/htbc/sdfs/%s.sdf" %f for f in df.index.values], parallel=True, worker_pool=worker_pool)
    smiles_list = [s.strip() for s in smiles_list]
    pc_smiles = [s.strip() for s in pc_smiles]
    new_df = pd.DataFrame(list(zip(smiles_list, compound_names, pc_smiles, pc_names, pc_ids)), index=df.index, columns=["smiles", "pubchem_name", "pubchem_smiles", "alternate_name", "pubchem_ID"])
    return(new_df)

In [10]:
import multiprocessing as mp
mp.cpu_count()

12

In [11]:
from ipyparallel import Client
rc = Client()
print(len(rc.ids))
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

150


<AsyncMapResult: chdir>

In [12]:
#import grids
#reload(grids)
from grids import *

precision = "SP"

ref_receptors_dir = "/home/enf/md_simulations/MOR/bu72_ref_receptors/conformation"
active_ref_dir = "%s/bu72_ionized_pymol_RL_conformation.pdb" %ref_receptors_dir
inactive_ref_dir = "%s/bu72_aligned_4DKL_R_L_conformation.pdb" %ref_receptors_dir

analysis_dir = "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/analysis_n_clusters25_random"
base_docking_dir = "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components10all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_regularization_wolf_autoShrinkage0pt01-backup/docking_100clusters"
ligands_dir = "/home/enf/htbc/sdfs/32_stereoisomers"
grid_dir = "%s/grids" %base_docking_dir
docking_dir = "%s/docking_%s" %(base_docking_dir, precision)
docking_dir = "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/all_clusterer_25clusters_1samples_samples_kdtree/docking_SP_32-stereoisomers_6-ring-conf/htbc"
#docking_dir = "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/all_clusterer_25clusters_1samples_samples_kdtree/docking_XP"

In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
import analysis
from importlib import reload
reload(analysis)
from analysis import *

docking_df, poses_df = analyze_docking_results_multiple(docking_dir, precision, "%s/summary.pkl" %docking_dir,
                                                        poses_summary=None, redo=False, reread=False,
write_to_disk=True, worker_pool=None, parallel=True)

#docking_df, poses_df = analyze_docking_results_in_dir(docking_dir, ligands_dir, write_to_disk=True, redo=True)
#docking_df[docking_df.columns] = np.nan_to_num(docking_df[docking_df.columns].values)
zeros_per_row = (docking_df == 0).sum(axis=1)
docking_df = docking_df.loc[zeros_per_row < 5]
#where_zeros = np.where(docking_df.values == 0)
#for row_index in where

#keep_cols = []
#for drug in docking_df.index:
#    if np.where(docking_df.loc[column].values == 0).shape[0] < 3:
#        keep_cols.append(col)
#docking_df = docking_df[keep_cols]


In [15]:
docking_df.iloc[:5]

,bu72_aligned_4DKL_R_L_conformation,bu72_ionized_pymol_RL_conformation,cluster0_sample0,cluster10_sample0,cluster11_sample0,cluster12_sample0,cluster13_sample0,cluster14_sample0,cluster15_sample0,cluster16_sample0,...,cluster23_sample0,cluster24_sample0,cluster2_sample0,cluster3_sample0,cluster4_sample0,cluster5_sample0,cluster6_sample0,cluster7_sample0,cluster8_sample0,cluster9_sample0
_99749,6.07,5.81,5.60,6.81,5.62,6.59,5.35,6.34,5.82,6.53,...,4.21,5.56,6.09,8.30,5.07,6.35,7.06,6.29,6.95,5.92
_79828,6.87,6.47,5.84,6.33,6.76,6.38,5.28,5.37,5.78,6.13,...,5.00,5.89,5.31,6.50,5.28,6.61,5.44,6.52,5.88,5.79
_71061,4.73,6.46,5.54,6.06,6.13,6.62,5.39,3.66,4.24,6.04,...,4.27,5.65,4.39,6.52,4.83,4.92,4.94,5.60,4.92,5.64
_112476,4.88,6.95,5.36,5.57,5.07,5.52,4.72,5.75,4.21,4.99,...,4.38,5.71,5.58,5.85,4.48,4.42,5.32,4.68,6.30,4.70
_21666,6.16,4.85,5.45,5.43,5.51,5.28,5.87,6.76,5.52,5.83,...,4.95,5.08,5.34,6.70,4.98,4.23,5.40,5.28,6.42,5.87


In [16]:
full_docking_df = copy.deepcopy(docking_df)
#full_docking_df = pd.concat([ref_df, docking_df[[c for c in docking_df.columns.values.tolist() if "cluster" in c]]], axis=1)
full_docking_df[full_docking_df.columns] = np.nan_to_num(full_docking_df[full_docking_df.columns].values)
new_names =  [n.replace("cluster", "State ").replace("_sample0", "") for n in full_docking_df.columns.values.tolist()]
new_names = ["Inactive Crystal", "Active Crystal"] + new_names[2:]
full_docking_df.columns = new_names
full_docking_df.index = [n.lower().strip().replace(" ", "_").replace("β", "beta").replace("α", "alpha").replace("Α", "alpha") for n in full_docking_df.index.values]
full_docking_df = full_docking_df.groupby(full_docking_df.index).first()

In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)

In [18]:
def add_difference_columns(df):
    new_df = copy.deepcopy(df)
    for i, col_i in enumerate(df.columns):
        print(i)
        for j, col_j in enumerate(df.columns):
           if j <= i: continue
    ##        if "crystal" in col_j.lower() or "crystal" in col_i.lower(): continue
           new_df["%s-%s" %(col_j, col_i)] = new_df[col_j].subtract(new_df[col_i])
    for i, col_i in enumerate(df.columns):
        if col_i != "Active Crystal":
            new_df["Active Crystal-%s" %col_i] = new_df["Active Crystal"].subtract(new_df[col_i])
    return(new_df)

In [19]:
agonist_model_dir = "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/analysis_n_clusters25_random/agonism_refined-wikipedia_SP_1000trees.pkl"
with open(agonist_model_dir, "rb") as f:
    agonist_model = pickle.load(f)

In [20]:
X_df = normalize_per_ligand(full_docking_df)
C_df = X_df[[c for c in X_df.columns if "state" not in c.lower()]]

In [25]:
agonism_test_df = pd.DataFrame(agonist_model["MSM Docking"][0].predict_proba(X_df), index=X_df.index, columns=["antagonist", "agonist"])
agonism_test_df.sort("agonist", ascending=False, inplace=True)

In [22]:
agonism_test_df = pd.DataFrame(agonist_model["Crystal Structures"][0].predict_proba(C_df), index=C_df.index, columns=["antagonist", "agonist"])
agonism_test_df.sort("agonist", ascending=False, inplace=True)

ValueError: Number of features of the model must  match the input. Model n_features is 3 and  input n_features is 8 

In [26]:
agonism_test_df.iloc[:10]

,antagonist,agonist
_49729,0.001,0.999
_63869,0.001,0.999
_49262,0.002,0.998
_34540,0.002,0.998
_99912,0.002,0.998
_7800,0.002,0.998
_115411,0.002,0.998
_48133,0.002,0.998
_53682,0.002,0.998
_47132,0.002,0.998


In [27]:
from efficacy_scripts import *
keep_top = 100
smiles_list, compound_names, pc_smiles, pc_names, pc_ids = convert_sdfs_to_compounds(["/home/enf/htbc/sdfs/%s.sdf" %f for f in agonism_test_df.iloc[:keep_top].index.values], parallel=True, worker_pool=dview)
smiles_list = [s.strip() for s in smiles_list]
pc_smiles = [s.strip() for s in pc_smiles]
agonism_names_df = pd.DataFrame(list(zip(smiles_list, compound_names, pc_smiles, pc_names, pc_ids)), index=agonism_test_df.iloc[:keep_top].index, columns=["smiles", "pubchem_name", "pubchem_smiles", "alternate_name", "pubchem_ID"])

Getting SMILES from SDFs...
Done. Now getting compound names from SMILES...
Done. returning compound names.


In [28]:
agonism_names_df.iloc[:10]

,smiles,pubchem_name,pubchem_smiles,alternate_name,pubchem_ID
_49729,S1CCN(CC1)C1CCN(CC1)Cc1ccccc1,Ambcb5455818,C1CN(CCC1N2CCSCC2)CC3=CC=CC=C3,Ambcb5455818,30982700
_63869,O=C(C1C(C1(C)C)C=C(Cl)Cl)NCCc1ccccc1,AG-690/10514039,CC1(C(C1C(=O)NCCC2=CC=CC=C2)C=C(Cl)Cl)C,AG-690/10514039,2851986
_49262,CCN(CCN(C)C)CCC(c1ccccc1)C,Ambcb5431960,CCN(CCC(C)C1=CC=CC=C1)CCN(C)C,Ambcb5431960,2846211
_34540,CCOc1cc(ccc1OC)CN1CCN(CC1)c1ccccc1OC,AC1LLIFE,CCOC1=C(C=CC(=C1)CN2CCN(CC2)C3=CC=CC=C3OC)OC,AC1LLIFE,1067955
_99912,CCOC(=O)c1[nH]c(c(c1C)C(=O)NC1CCCC(C1C)C)C,MolPort-007-720-339,CCOC(=O)C1=C(C(=C(N1)C)C(=O)NC2CCCC(C2C)C)C,MolPort-007-720-339,22515588
_7800,N#CSCC(=O)N1c2ccccc2C(CC1(C)C)(C)c1ccccc1,"2-oxo-2-(2,2,4-trimethyl-4-phenyl-3,4-dihydroquinolin-1(2H)-yl)ethyl thiocyanate",CC1(CC(C2=CC=CC=C2N1C(=O)CSC#N)(C)C3=CC=CC=C3)C,"2-oxo-2-(2,2,4-trimethyl-4-phenyl-3,4-dihydroquinolin-1(2H)-yl)ethyl thiocyanate",2886786
_115411,c1ccc(cc1)CN1CCC(CC1)n1cccc1,1-benzyl-4-(1H-pyrrol-1-yl)piperidine,C1CN(CCC1N2C=CC=C2)CC3=CC=CC=C3,1-benzyl-4-(1H-pyrrol-1-yl)piperidine,2779264
_48133,OCCN1CCN(CC1)CCC1=C(C)CCCC1(C)C,AC1O5D3Q,CC1=C(C(CCC1)(C)C)CCN2CCN(CC2)CCO,AC1O5D3Q,6456230
_53682,OCCN(Cc1ccccc1)CCC(c1ccccc1)C,Ambcb5421554,CC(CCN(CCO)CC1=CC=CC=C1)C2=CC=CC=C2,Ambcb5421554,45927593
_47132,CN(Cc1cccc(c1Cl)Cl)CCc1ccccn1,Ambcb5345874,CN(CCC1=CC=CC=N1)CC2=C(C(=CC=C2)Cl)Cl,Ambcb5345874,23826443


In [29]:
affinity_model_dir = "/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/analysis_n_clusters25_random/affinity_model_pIC50-8-8_chembl_none-removed_rfr_1000trees.pkl"
with open(affinity_model_dir, "rb") as f:
    affinity_model = pickle.load(f)

In [30]:
X_df = full_docking_df.copy()
C_df = X_df[[c for c in X_df.columns if "state" not in c.lower()]]

In [31]:
affinity_test_df = pd.DataFrame(affinity_model["MSM Docking"][0].predict_proba(X_df), index=X_df.index, columns=["non-binder", "binder"])
affinity_test_df.sort("binder", ascending=False, inplace=True)

In [32]:
affinity_test_df.iloc[:25]

,non-binder,binder
_116572,0.175,0.825
_48420,0.202,0.798
_7320,0.202,0.798
_14,0.205,0.795
_24056,0.207,0.793
_60769,0.215,0.785
_76651,0.216,0.784
_77940,0.218,0.782
_112636,0.223,0.777
_33051,0.226,0.774


In [35]:
affinity_test_df = pd.DataFrame(affinity_model["Crystal Structures"][0].predict_proba(C_df), index=C_df.index, columns=["non-binder", "binder"])
affinity_test_df.sort("binder", ascending=False, inplace=True)

In [33]:
affinity_names_df = create_named_df(affinity_test_df.iloc[:100], dview)

Getting SMILES from SDFs...
Done. Now getting compound names from SMILES...
Done. returning compound names.


In [106]:
affinity_names_df

,smiles,pubchem_name,pubchem_smiles,alternate_name,pubchem_ID
_116572,CNCc1cc(ccc1N1CCOCC1)C(F)(F)F,886851-52-1,CNCC1=C(C=CC(=C1)C(F)(F)F)N2CCOCC2,886851-52-1,18525888
_76651,OCCC1CNCCN1Cc1ccc(cc1)OCC,2-[1-(4-ethoxybenzyl)-2-piperazinyl]ethanol,CCOC1=CC=C(C=C1)CN2CCNCC2CCO,2-[1-(4-ethoxybenzyl)-2-piperazinyl]ethanol,2989709
_48420,CC1CCC(C1)N1CCN(CC1)c1ccc(cc1)Cl,AC1O5D6K,CC1CCC(C1)N2CCN(CC2)C3=CC=C(C=C3)Cl,AC1O5D6K,6456264
_33051,NCCC1(CCOC(C1)(C)C)c1ccc(cc1)F,"2-[4-(4-Fluoro-phenyl)-2,2-dimethyl-tetrahydro-pyran-4-yl]-ethylamine",CC1(CC(CCO1)(CCN)C2=CC=C(C=C2)F)C,"2-[4-(4-Fluoro-phenyl)-2,2-dimethyl-tetrahydro-pyran-4-yl]-ethylamine",2835263
_71678,COc1ccc(cc1)C1OC(=NN1C(=O)C)c1ccc(cc1)Cl,AKOS003363717,CC(=O)N1C(OC(=N1)C2=CC=C(C=C2)Cl)C3=CC=C(C=C3)OC,AKOS003363717,17380573
_14,COc1cc(CN2CCNCC2)cc2c1OCO2,"1-[(7-methoxy-1,3-benzodioxol-5-yl)methyl]piperazine",COC1=CC(=CC2=C1OCO2)CN3CCNCC3,"1-[(7-methoxy-1,3-benzodioxol-5-yl)methyl]piperazine",3128130
_112636,COc1cc(OC)ccc1C1C=C(Nc2n1nnn2)c1ccc(cc1)C,AC1NXUBE,CC1=CC=C(C=C1)C2=CC(N3C(=N2)N=NN3)C4=C(C=C(C=C4)OC)OC,AC1NXUBE,5792042
_121896,CC(=O)OC1/C=C(/C)\CCC2C(CC3C1C(=C)C(=O)O3)(C)O2,Chrysanthin,CC1=CC(C2C(CC3(C(O3)CC1)C)OC(=O)C2=C)OC(=O)C,Chrysanthin,5860420
_47478,COc1cc(CN(C2CCN(CC2)C)C)cc(c1)OC,Oprea1_288259,CN1CCC(CC1)N(C)CC2=CC(=CC(=C2)OC)OC,Oprea1_288259,23885101
_24056,COc1ccc(cc1)CCC(C1(O)CCCCC1)O,ST069094,COC1=CC=C(C=C1)CCC(C2(CCCCC2)O)O,ST069094,4375993


In [34]:
affinity_names_df

,smiles,pubchem_name,pubchem_smiles,alternate_name,pubchem_ID
_116572,CNCc1cc(ccc1N1CCOCC1)C(F)(F)F,886851-52-1,CNCC1=C(C=CC(=C1)C(F)(F)F)N2CCOCC2,886851-52-1,18525888
_48420,CC1CCC(C1)N1CCN(CC1)c1ccc(cc1)Cl,AC1O5D6K,CC1CCC(C1)N2CCN(CC2)C3=CC=C(C=C3)Cl,AC1O5D6K,6456264
_7320,OC(CN1C(C)(C)CCCC1(C)C)COc1ccccc1,"1-phenoxy-3-(2,2,6,6-tetramethylpiperidin-1-yl)propan-2-ol",CC1(CCCC(N1CC(COC2=CC=CC=C2)O)(C)C)C,"1-phenoxy-3-(2,2,6,6-tetramethylpiperidin-1-yl)propan-2-ol",211983
_14,COc1cc(CN2CCNCC2)cc2c1OCO2,"1-[(7-methoxy-1,3-benzodioxol-5-yl)methyl]piperazine",COC1=CC(=CC2=C1OCO2)CN3CCNCC3,"1-[(7-methoxy-1,3-benzodioxol-5-yl)methyl]piperazine",3128130
_24056,COc1ccc(cc1)CCC(C1(O)CCCCC1)O,ST069094,COC1=CC=C(C=C1)CCC(C2(CCCCC2)O)O,ST069094,4375993
_60769,C=CCOc1ccc2c(c1)c(C(=O)NCc1ccccc1C(F)(F)F)c(n2Cc1ccco1)C,,,,
_76651,OCCC1CNCCN1Cc1ccc(cc1)OCC,2-[1-(4-ethoxybenzyl)-2-piperazinyl]ethanol,CCOC1=CC=C(C=C1)CN2CCNCC2CCO,2-[1-(4-ethoxybenzyl)-2-piperazinyl]ethanol,2989709
_77940,O=C(c1ccccc1)ON1C(C)(C)CC(CC1(C)C)NS(=O)(=O)C,ST048187,CC1(CC(CC(N1OC(=O)C2=CC=CC=C2)(C)C)NS(=O)(=O)C)C,ST048187,1242519
_112636,COc1cc(OC)ccc1C1C=C(Nc2n1nnn2)c1ccc(cc1)C,AC1NXUBE,CC1=CC=C(C=C1)C2=CC(N3C(=N2)N=NN3)C4=C(C=C(C=C4)OC)OC,AC1NXUBE,5792042
_33051,NCCC1(CCOC(C1)(C)C)c1ccc(cc1)F,"2-[4-(4-Fluoro-phenyl)-2,2-dimethyl-tetrahydro-pyran-4-yl]-ethylamine",CC1(CC(CCO1)(CCN)C2=CC=C(C=C2)F)C,"2-[4-(4-Fluoro-phenyl)-2,2-dimethyl-tetrahydro-pyran-4-yl]-ethylamine",2835263


In [35]:
keep = 1000
intersecting_ligands = list(set(affinity_test_df.index.values.tolist()[:keep]).intersection(set(agonism_test_df.index.values.tolist()[:keep])))

In [36]:
len(intersecting_ligands)

3

In [ ]:
names_df = create_named_df(combined_df, None)

Getting SMILES from SDFs...
Done. Now getting compound names from SMILES...
Done. returning compound names.


In [ ]:
names_df

,smiles,pubchem_name,pubchem_smiles,alternate_name,pubchem_ID
_59621,CCOc1ccc2c(c1)c(C(=O)N1CCN(CC1)CC=Cc1ccccc1)c(n2CCc1ccc(cc1)OC)C,,,,
_2488,COc1c(ccc(c1OC)OC)CN1CCN(CC1)Cc1ccccc1C(F)(F)F,AK-968/13149944,COC1=C(C(=C(C=C1)CN2CCN(CC2)CC3=CC=CC=C3C(F)(F)F)OC)OC,AK-968/13149944,1126253
_44195,CCCn1c(nc2c1cccc2)C(c1ccc(cc1)N(C)C)O,CBMicro_014544,CCCN1C2=CC=CC=C2N=C1C(C3=CC=C(C=C3)N(C)C)O,CBMicro_014544,3039564
_99190,COc1ccccc1CNC(=O)C1CCCN(C1)Cc1cnn(c1n1cccc1)c1ccccc1,AKOS001893969,COC1=CC=CC=C1CNC(=O)C2CCCN(C2)CC3=C(N(N=C3)C4=CC=CC=C4)N5C=CC=C5,AKOS001893969,22511744
_52376,NS(=O)(=O)c1ccc2c(c1)C1C=CCC1C(N2)c1cccc2c1cccc2,CHEMBL412119,C1C=CC2C1C(NC3=C2C=C(C=C3)S(=O)(=O)N)C4=CC=CC5=CC=CC=C54,CHEMBL412119,5038679
_123473,CCN(Cc1c(O)ccc2c1cccc2)Cc1cc(Cl)cc(c1O)C,NSC88850,CCN(CC1=C(C(=CC(=C1)Cl)C)O)CC2=C(C=CC3=CC=CC=C32)O,NSC88850,259173
_94499,OC(Cn1cc(c2c1cccc2)/C=C/C(=O)c1ccc(cc1)Br)CN1CCOCC1,AE-848/42799179,C1COCCN1CC(CN2C=C(C3=CC=CC=C32)C=CC(=O)C4=CC=C(C=C4)Br)O,CTK7J6616,11835319
_88928,CC(=O)C(n1c(CN(Cc2ccccc2)Cc2ccccc2)nc2c1c(=O)[nH]c(=O)n2C)C,AC1NL9P4,CC(C(=O)C)N1C(=NC2=C1C(=O)NC(=O)N2C)CN(CC3=CC=CC=C3)CC4=CC=CC=C4,AC1NL9P4,4976490
_49845,c1ccc(cc1)CCN1CCC(CC1)NC(c1ccccc1)Cc1ccccc1,AC1ME1BV,C1CN(CCC1NC(CC2=CC=CC=C2)C3=CC=CC=C3)CCC4=CC=CC=C4,AC1ME1BV,2846316
_56627,O=C(N1N=CCC1(CCOCc1ccccc1)C(=O)NC1CCc2c1cccc2)NCc1ccccc1,,,,


In [35]:
combined_df = pd.concat([affinity_test_df, agonism_test_df], axis=1)
combined_df["joint"] = combined_df["agonist"].multiply(combined_df["binder"])
combined_df.sort("joint", inplace=True, ascending=False)
combined_df.iloc[:20]

,non-binder,binder,antagonist,agonist,joint
_7320,0.202,0.798,0.058,0.942,0.751716
_77940,0.218,0.782,0.047,0.953,0.745246
_115830,0.245,0.755,0.045,0.955,0.721025
_42633,0.284,0.716,0.011,0.989,0.708124
_40201,0.279,0.721,0.026,0.974,0.702254
_121907,0.256,0.744,0.065,0.935,0.695640
_37605,0.294,0.706,0.052,0.948,0.669288
_102267,0.330,0.670,0.003,0.997,0.667990
_53765,0.317,0.683,0.023,0.977,0.667291
_121545,0.259,0.741,0.101,0.899,0.666159


In [36]:
names_df = create_named_df(combined_df.iloc[:100], dview)

Getting SMILES from SDFs...
Done. Now getting compound names from SMILES...
Done. returning compound names.


In [37]:
names_df

,smiles,pubchem_name,pubchem_smiles,alternate_name,pubchem_ID
_7320,OC(CN1C(C)(C)CCCC1(C)C)COc1ccccc1,"1-phenoxy-3-(2,2,6,6-tetramethylpiperidin-1-yl)propan-2-ol",CC1(CCCC(N1CC(COC2=CC=CC=C2)O)(C)C)C,"1-phenoxy-3-(2,2,6,6-tetramethylpiperidin-1-yl)propan-2-ol",211983
_77940,O=C(c1ccccc1)ON1C(C)(C)CC(CC1(C)C)NS(=O)(=O)C,ST048187,CC1(CC(CC(N1OC(=O)C2=CC=CC=C2)(C)C)NS(=O)(=O)C)C,ST048187,1242519
_115830,NCc1ccccc1OC1CCOCC1,898289-33-3,C1COCCC1OC2=CC=CC=C2CN,898289-33-3,18525860
_42633,CCC(=O)OC1(CCN(C(C1C)C)C)c1ccccc1,CBDivE_014579,CCC(=O)OC1(CCN(C(C1C)C)C)C2=CC=CC=C2,CBDivE_014579,6612424
_40201,CN(CC1CCCCC1(OC(=O)C)c1ccccc1)C,AC1LC34P,CC(=O)OC1(CCCCC1CN(C)C)C2=CC=CC=C2,AC1LC34P,546980
_121907,COc1ccc2c(c1O)[C@@]13CCN([C@H](C2)[C@@H]3CCC(=O)C1)C,Dihydrothebainone,CN1CCC23CC(=O)CCC2C1CC4=C3C(=C(C=C4)OC)O,AE-562/12222547,5355412
_37605,Cc1ccc(cc1)OCCCCCCc1c(C)n[nH]c1C,AG-664/14117620,CC1=CC=C(C=C1)OCCCCCCC2=C(NN=C2C)C,AG-664/14117620,2057791
_102267,CN1CCN(CC1)Cc1csc2c1CCCC2,ZINC20514305,CN1CCN(CC1)CC2=CSC3=C2CCCC3,ZINC20514305,22554311
_53765,CN(CCCN1C(=O)CSC1c1ccc(cc1)Br)C,STK386756,CN(C)CCCN1C(SCC1=O)C2=CC=C(C=C2)Br,STK386756,2861590
_121545,COc1c(ccc(c1OC)OC)CN1CCNCC1,TRIMETAZIDINE,COC1=C(C(=C(C=C1)CN2CCNCC2)OC)OC,TRIMETAZIDINE,21109


In [38]:
docking_dir

'/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/all_clusterer_25clusters_1samples_samples_kdtree/docking_SP_32-stereoisomers_6-ring-conf/htbc'

In [ ]:
test_df[["pubchem_name", "smiles", "alternate_smiles", "alternate_name"]] = compound_names_df.loc[test_df.index][["pubchem_name", "smiles", "pubchem_smiles", "alternate_name"]]
bret_pred_df = test_df.loc[[n for n in test_df.index.values if n in common_ligands]]
bret_pred_df["label"] = 0.
bret_pred_df["label"].loc[common_ligands] = y
from rdkit.ML.Scoring.Scoring import CalcAUC
CalcAUC(bret_pred_df.values[:,(0,1)], col=0)#, alpha=10)

In [ ]:
test_df.loc[[n for n in test_df.index.values if "compd0" in n or "compd11" in n]].sort("class B", ascending=False,inplace=False).iloc[:]